File: prices.ipynb\
Author: alexkobz\
Date: 08-10-2024\
Task: DSRFU-298

#### Шаг 0. Импорт и получение токена

In [1]:
import asyncio
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt, date, timedelta
from functions.divide_chunks import divide_chunks
from math import ceil
from functions.get_date import last_day_month, last_day_month_str, last_work_date_month, last_work_date_month_str

# последний день месяца
REPORT_DATE = last_day_month
REPORT_DATE_STR = last_day_month_str
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)

In [2]:
from rudata.RuDataDF import RuDataDF
from rudata.RuDataRequest import RuDataRequest
RuDataRequest.set_headers()

Getting token started
Getting token finished


In [62]:
month_year = REPORT_DATE.strftime("%m.%Y")
isin_file = f"ISIN_{month_year}.xlsx"
Prices = pd.read_excel(f"./data/Input/{isin_file}", names=["ISIN"]).dropna().set_index("ISIN")
Prices["Date"] = REPORT_DATE
Prices[["Name", "Type", "Principal", "Currency", "Clean price pct", "Clean price", "Interest", "Price", 
        "Source", "ISIN in DB", "FX Rate", "RUB Price", "CBR discount 6D", "Discount NCC", "NCC collateral"]] = np.nan

#### Шаг 1. RuData

##### Шаг 1.0. RuData stage

In [4]:
FintoolReferenceData = RuDataDF("FintoolReferenceData").df
FintoolReferenceData.head()

FintoolReferenceData started
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
1500, 0
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
1500, 1500
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
1500, 3000
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
1500, 4500
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
1500, 6000
post start
post start
post start
post start


,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,current_coupontype,profit_type,counter,rn
0,2.0,2.0,2.0,2.0,2.0,None,2.0,0.0,2.0,2.0,2.0,2.0,2.0,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,S111,None,Старшие необеспеченные,4000,None,None,"МинФин РФ, государственные краткосрочные беску...",None,None,ГКО-21134,NaN,21134RMFS,None,МинФин,0.0,act/365,None,None,Дисконтный,Облигация,None,Необеспеченные,0.0,0.0,0.0,Краткосрочные,None,1.0,RU,0.0,RUB,1000.0,NaN,0.0,0.0,Погашен,2.500000e+09,2500000.0,2.500000e+09,2500000.0,Минфин РФ,00013474,Государственные,6874.0,7710168360,RU,г.Москва,Минфин РФ,00013474,Государственные,6874.0,7710168360,RU,г.Москва,None,0.0,1999-12-17T00:00:00,None,110.0,1999-12-17T00:00:00,2000-04-05T00:00:00,0.0,1999-12-17T00:00:00,Гос,Классические,Постоянный,2023-04-19T15:55:00,NaN,Нет информации,0.0,None,NaN,None,NaN,False,False,False,2.0,NaN,True,None,0.0,False,False,None,RU,1000,1000,NaN,False,NaN,False,False,None,416.0,NaN,None,None,100106,1
1,3.0,3.0,3.0,4.0,3.0,SU21135RMFS7,3.0,1.0,2.0,3.0,2.0,3.0,2.0,Государственные краткосрочные бескупонные обли...,Финансовые органы,None,135,0.0,0.0,None,0.0,0.0,1000.0,RUB,1500000.0,21135RMFS,1999-12-10T00:00:00,Финансовые орга

In [5]:
AccruedInterestOnDate = RuDataDF("AccruedInterestOnDate").df
AccruedInterestOnDate.head()

AccruedInterestOnDate started
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
500, 0
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
500, 500
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
500, 1000
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
500, 1500
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
500, 2000
post start
post start
post start
post start
post 

,fintoolId,shiftDays,shiftTypeSource,typeCurrency,date,accruedInterest,accruedInterestPct,currentFaceValue,currentFaceValuePct,calculationType,error,faceValueCurrency,accruedInterestCurrency,faceValueLastKnownDate
0,4490,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '4490': Нет...,None,None,None
1,4478,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '4478': Нет...,None,None,None
2,4464,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '4464': Нет...,None,None,None
3,4474,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '4474': Нет...,None,None,None
4,4454,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '4454': Нет...,None,None,None


In [6]:
RUPriceHistory = RuDataDF("RUPriceHistory").df
RUPriceHistory.shape

RUPriceHistory started
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
4788, 0
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
4809, 4788
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
4804, 9597
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
4789, 14401
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
4813, 19190
post start
post start
post start
post start
post

(39916, 82)

In [7]:
ExchangeTree = RuDataDF("ExchangeTree").df
ExchangeTree.head()

ExchangeTree started
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
238, 0
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
0, 238
ExchangeTree finished. ExchangeTree shape (238, 10)


,id,id_parent,shortname_rus,shortname_eng,fullname_rus,fullname_eng,sort_order,update_date,counter,rn
0,144,63,Нал,Cash,Наличная валюта,Cash,0,2020-12-28T17:03:09,238,1
1,270,63,RUDIP 2,RUDIP 2,RUDIP 2,RUDIP 2,1,2022-10-31T19:41:24,238,2
2,84,83,США,US,Макроиндикаторы США,USA macroindicators,1,2020-12-28T17:03:09,238,3
3,85,83,РФ,RU,Макроиндикаторы Россия,Russia macroindicators,2,2020-12-28T17:03:09,238,4
4,2,1,РТС,RTS,РТС индексы,RTS indexes,2,2020-12-28T17:03:09,238,5


In [24]:
MoexSecurities = RuDataDF("MoexSecurities").df
MoexSecurities.head()

MoexSecurities started
post start
post start
post start
post start
post start
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
post processing
post processing
sleep 1
sleep 1
1500, 0
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
1500, 1500
post start
post start
post start
post start
post start
post processing
post processing
post processing
sleep 1
sleep 1
post processing
post processing
sleep 1
sleep 1
sleep 1
1500, 3000
post start
post start
post start
post start
post start
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
post processing
post processing
sleep 1
sleep 1
1500, 4500
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
1500, 6000
post start
post start
post start
post start
post s

,id_iss,secid,boardid,board_title,board_group,board_group_title,market,engine,short_name,isin,cfi_ifx,cfi_name,lot_size,expiration_date,currency,emitent_code,emitent_name,market_sector,min_price_increment,fintoolid,reg_code,update_date,counter,rn
0,105013,RU000A0JPEZ2,TQIF,Т+: Паи - безадрес.,stock_shares_tplus,Т+: Основной режим - безадрес.,shares,stock,СибиряК-ЖилНедв-З-Н,RU000A0JPEZ2,EUCLXX,Паи / Тип фонда / Закрытый ПИФ,1000,None,RUB,SIBK,СибиряК УК,Финанс.сервисы,NaN,23674.0,0678-75408061,2024-03-25T13:20:04.633517,197756,901
1,105077,RU000A0JPD64,EQOB,Т0 Облигации - безадрес.,stock_bonds,Т0: Основной режим - безадрес.,bonds,stock,ТрансФинМ-02,RU000A0JPD64,BCCXXX,Облигации / Сектор / Корпоративные,1,2012-08-10T00:00:00,RUB,TRFM,ТрансФин-М,Фин.сервис - Лизинг,NaN,19225.0,4-02-36182-R,2024-09-20T13:20:03.739634,197756,902
2,105187,RU000A0JPF54,TQIF,Т+: Паи - безадрес.,stock_shares_tplus,Т+: Основной режим - безадрес.,shares,stock,Пиоглбл-ФдНедв2й-З-Н,None,EUCLXX,Паи / Тип фонда / Закрытый ПИФ,1,None,RUB,PGLM,ПТК УК,Финанс.сервисы,NaN,NaN,None,2023-08-01T14:45:22.563629,197756,903
3,105188,RU000A0JPCL6,EQDB,Основной режим: Облигации Д - безадрес.,stock_bonds,Т0: Основной режим - безадрес.,bonds,stock,Держава-Ф,RU000A0JPCL6,BCCXXX,Облигации / Сектор / Корпоративные,1,2010-07-27T00:00:00,RUB,XX008470,Держава-Финанс,Финанс.сервисы,NaN,21369.0,4-01-36175-R,2022-11-01T12:34:30.440279,197756,904
4,105304,RU000A0JPC73,EQOB,Т0 Облигации - безадрес.,stock_bonds,Т0: Основной режим - безадрес.,bonds,stock,КубаньЭн-01,RU000A0JPC73,BCCXXX,Облигации / Сектор / Корпоративные,1,2010-07-21T00:00:00,RUB,KUBE,Россети Кубань,Электроэнергетика,NaN,26916.0,4-01-00063-A,2023-08-29T15:24:23.573185,197756,905


In [30]:
%%time
import aiohttp
import socket
from typing import List, Union, Optional, Dict, Any
from rudata import DocsAPI

isins = Prices[Prices["Price"].isna()].index.unique().tolist()
key = "EndOfDay"
url: str = getattr(DocsAPI, key).url
payload: Dict[str, Any] = getattr(DocsAPI, key)().payload()
payloads = []
for isin in isins:
    payload["isin"] = isin
    payloads.append(payload.copy())
    
async def end_of_day_rudata():
    end_of_day = []
    async with aiohttp.ClientSession(
            connector=aiohttp.TCPConnector(limit=DocsAPI.LIMIT,
                                           family=socket.AF_INET),
            trust_env=True,
            timeout=aiohttp.ClientTimeout(3600)
    ) as session:
        for i, chunk in enumerate(divide_chunks(payloads, DocsAPI.LIMIT)):
            tasks: List[asyncio.Task] = [asyncio.create_task(
                RuDataRequest(url, session).post(payload=payload)
            ) for payload in chunk]
            resAll: List[List[dict]] = [await task for task in asyncio.as_completed(tasks, timeout=600)]
            result: List[dict] = [row for task_res in resAll for row in task_res]
            end_of_day.extend(result)
    return end_of_day
    
end_of_day = asyncio.run(end_of_day_rudata())
EndOfDay = pd.DataFrame(end_of_day)
EndOfDay.head()

post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 

,accruedint,admquote,akm_bl_n,akm_e_n,ask,ask_size,ask_size_total,avge_chng,avge_chng_pct,avge_prce,avge_yield,bid,bid_size,bid_size_total,boardid,boardname,buybackdate,buybackprice,cbr_lombard,cbr_pledge,chng,chng_m,chng_m_pct,chng_pct,chng_w,chng_w_pct,chng_y,chng_y_pct,close,consensus_num_est,consensus_time,convexity,convexity_agg,convexity_o,cpn,cpn_date,cpn_period,cpprice,deal_acc,discount,dnt06beg,dnt06max,dnt06min,dnt120beg,dnt120max,dnt120min,dnt14beg,dnt14max,dnt14min,dnt200beg,dnt200max,dnt200min,dnt365beg,dnt365max,dnt365min,duration,duration_agg,duration_n,duration_o,duration_on,exch,exp_bl_n,exp_e_n,facevalue,facevalue_currency,fact_frcst_chng,fch_b_i,fch_b_n,fch_e_dlti,fch_e_lti,forecast,frcst_chng,high,ifx_b_i,ifx_b_n,ifx_e_i,ifx_e_n,isin,issuer,issuesize,last,last_time,last_yield,last2avge,lastsettlecode,lclose,lnkcurrency,lotsize,low,mat_date,mat_date_dt,mcap,mds_b_i,mds_b_n,mds_e_lti,mds_e_ltn,mprice,mprice2,name,open,...,pvbp_o,rec_buy_pct,rec_change,rec_hold_pct,rec_sell_pct,rec_val,recommendation,rus_b_n,rus_e_i,rus_e_n,seccode,secname,secname_e,sector,settlecode,shortname_eng,shortname_rus,snp_b_i,snp_b_n,snp_e_lti,snp_e_n,spread,status,target_price,tick_acc,time,tp_chng,tp_chng_m,tp_chng_m_pct,tp_chng_pct,tp_chng_q,tp_chng_q_pct,tp_chng_time,tp_chng_y,tp_chng_y_pct,tp_deviation,tp_high,tp_low,tp_median,val,val_acc,val_acc_usd,val_usd,vol,vol_acc,vol_lots,y2o_ask,y2o_bid,y2o_change,y2o_change_pct,y2o_last,y2o_prev,y2o_wap,yield_2,yield_agg,yield_ask,yield_average_12m,yield_average_1m,yield_average_2m,yield_average_2w,yield_average_3m,yield_average_6m,yield_average_o12m,yield_average_o1m,yield_average_o2m,yield_average_o2w,yield_average_o3m,yield_average_o6m,yield_bid,yield_chng,yield_cpprice,yield_cwa,yield_high,yield_low,yield_o_cwa,yield_o_swa,yield_open,yield_prev,yield_swa,total_return,full_price_pct,zspread,gspread,currency,fintoolId,id_iss,lcurrentprice,prevlclose,lcp_chng,lcp_chng_pct,bid_price_change,ask_price_change,bid_yield_change,ask_yield_change,id_trade_site,ifrs_level,rd_exch,rd_score,add_date,update_date
0,NaN,None,None,None,253.94,None,0.0,-2.63,-1.01,256.14,NaN,253.91,None,0.0,TQBR,T+ Акции и ДР,None,NaN,NaN,None,-6.12,-32.30,-11.28,-2.35,-7.41,-2.83,-11.23,-4.23,260.06,None,None,NaN,NaN,NaN,NaN,None,NaN,None,20954.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,MICEX,None,None,3.000000,None,None,None,None,None,None,None,None,261.30,None,None,None,None,RU0009029557,None,1.000000e+09,253.94,2024-08-30T23:49:58,NaN,-4.83,None,None,None,10.0,252.65,None,None,2.561400e+11,None,None,None,None,256.57,None,SBERP.MICEX,259.61,...,NaN,None,None,None,None,None,Покупать,None,None,None,SBERP,"Сбербанк России ОАО, АП - ФБ ММВБ","Sberbank Russia OJSC, PS - MICEX",None,None,Sberbank-PS,Сбербанк-АП,None,None,None,None,0.03,None,370.63,None,2024-08-30T00:00:00,None,None,None,None,None,None,None,None,None,None,None,None,None,1.109183e+09,1.109183e+09,12128850.77,12128850.77,4330360.0,4330360.0,433036.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,RUB,4443,101907,None,None,None,None,None,None,None,None,170,NaN,None,None,2024-08-30T23:55:01,2024-08-30T23:55:01
1,NaN,None,None,None,353.00,None,0.0,-7.00,-1.94,352.50,NaN,346.80,None,0.0,TQBR,T+ Акции и ДР,None,NaN,NaN,None,-15.00,-99.60,-22.33,-4.15,-19.60,-5.35,-382.60,-52.48,361.40,None,None,NaN,NaN,NaN,NaN,None,NaN,None,25285.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,MICEX,None,None,0.000451,None,None,None,None,None,None,None,None,365.40,None,None,None,None,RU000A106YF0,None,0.000000e+00,346.40,2024-08-30T23:49:54,NaN,-13.10,None,None,RUB,1.0,342.00,None,None,0.000000e+00,None,None,None,None,354.40,None,VK-1-ao.E.MICEX.TQBR,364.00,...,NaN,None,None,None,None,None,None,None,None,None,VKCO,"""ВК"", МКПАО, акции обыкновенные бездокументарн..."

In [31]:
%%time
import aiohttp
import socket
from typing import List, Union, Optional, Dict, Any
from rudata import DocsAPI
from time import sleep

currencies = pd.read_csv("./data/Input/currencies.csv", header=0, encoding='cp1251', sep=';', usecols=["Currency"])["Currency"].tolist()
key = "CurrencyRate"
url: str = getattr(DocsAPI, key).url
payload: Dict[str, Any] = getattr(DocsAPI, key)().payload()
currencies_rudata = []
for currency in currencies:
    payload["from"] = currency
    ans = requests.post(url, json={'from': currency, 'to': 'RUB', 'date': REPORT_DATE_STR}, headers=RuDataRequest.headers)
    result: dict = ans.json()
    result["currency"] = currency
    currencies_rudata.append(result)
    sleep(1)
CurrencyRate = pd.DataFrame(currencies_rudata)
CurrencyRate.head()

CPU times: user 5.09 s, sys: 393 ms, total: 5.49 s
Wall time: 11min 42s


,rate,endOfDayRate,error,currency
0,24.829600,24.829600,None,AED
1,NaN,NaN,None,AFN
2,NaN,NaN,None,ALL
3,NaN,NaN,None,KMF
4,0.235303,0.235303,None,AMD


##### Шаг 1.1. Discounts

###### Шаг 1.1.1. CBR

In [32]:
from sources.CBR_Soap import CBR_Soap

headers = {key: item for key, item in RuDataRequest.headers.items() if key != "Authorization"}
cbr_disc = CBR_Soap()\
.get_discounts(date=last_work_date_month_str)[['ISIN', 'от 1 месяца до 180 дней_Beg']]\
.rename(columns={'от 1 месяца до 180 дней_Beg': 'CBR discount 6D'})\
.set_index('ISIN')

GET http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?wsdl using urllib2 3.8
GET http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?wsdl using urllib2 3.8
GET http://www.cbr.ru/secinfo/secinfo.asmx?wsdl using urllib2 3.8
GET http://www.cbr.ru/secinfo/secinfo.asmx?wsdl using urllib2 3.8


In [63]:
Prices.update(cbr_disc)
Prices['CBR discount 6D'] = Prices['CBR discount 6D'].fillna(1)

###### Шаг 1.1.2. nationalclearingcentre Рыночные риски

In [61]:
currMarketRatesResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/fondMarketRates?action=xls&xls=1&lang=ru&date={last_work_date_month.strftime("%d.%m.%Y")}', headers=headers)
currMarketRates = pd.read_excel(currMarketRatesResponse.content,header=2)
currMarketRates = currMarketRates[(currMarketRates['Диапазон']==1)&(currMarketRates['Ставка риска падения цены,%']!=100)].reset_index(drop=True)
currMarketRates = currMarketRates\
.merge(MoexSecurities[['secid', 'isin']], how='left', left_on='Инструмент', right_on='secid')\
.rename(columns={'isin': 'ISIN'})\
.groupby('ISIN')\
.last()
currMarketRates = currMarketRates[currMarketRates.index.str.match('([A-Z]{2})([A-Z0-9]{9})([0-9]{1})$')]
currMarketRates = currMarketRates[(currMarketRates['Диапазон']==1)&(currMarketRates['Ставка риска падения цены,%']!=100)]
currMarketRates=currMarketRates[['Ставка риска падения цены,%']]\
.rename(columns={'Ставка риска падения цены,%': 'Risk rate'})
currMarketRates['Discount NCC'] = currMarketRates['Risk rate'].apply(lambda x: min(30, ceil(x/np.sqrt(2)))) #Формула из п.12.1 Методики НКЦ определения риск-параметров
currMarketRates /= 100

In [64]:
Prices.update(currMarketRates)
Prices['Discount NCC'] = Prices['Discount NCC'].fillna(1)

###### Шаг 1.1.3. nationalclearingcentre Параметры ценных бумаг

In [65]:
securInfoResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/securInfo?action=xls&lang=ru&settleDate={last_work_date_month.strftime("%d.%m.%Y")}', headers=headers)
securInfo = pd.read_excel(securInfoResponse.content, header=2)[['ISIN ЦБ', 'Принимается в обеспеч.']]\
.rename(columns={
    'ISIN ЦБ': 'ISIN',
    'Принимается в обеспеч.': 'NCC collateral'
})\
.dropna()\
.set_index('ISIN')
securInfo = securInfo[securInfo['NCC collateral']=='Да']

In [66]:
Prices.update(securInfo)
Prices['NCC collateral'] = Prices['NCC collateral'].fillna('Нет')

##### Шаг 1.2. RuData transform

In [67]:
TransformFintoolReferenceData = FintoolReferenceData[['isincode', 'nickname', 'fintooltype', 'fintoolid', 'faceftname']]\
.rename(columns={
    "isincode": "ISIN",
    "nickname": "Name",
    "fintoolid": "Fintool",
    "fintooltype": "Type", 
    "faceftname": "Currency"
})
TransformFintoolReferenceData["ISIN in DB"] = 1
TransformFintoolReferenceData["Source"] = 'RU Data (ЦЦ НРД)'
TransformFintoolReferenceData = TransformFintoolReferenceData.merge(
    AccruedInterestOnDate[['fintoolId', 'accruedInterest', 'currentFaceValue']], how="left", left_on="Fintool", right_on="fintoolId")
TransformFintoolReferenceData = TransformFintoolReferenceData\
.rename(columns={
    'accruedInterest': 'Interest', 
    'currentFaceValue': 'Principal'})\
.drop(columns=["fintoolId"])

In [68]:
RUPriceHistoryLast = RUPriceHistory.groupby('isincode').last().rename(columns={
    'vp_pct': 'Clean price pct', 
    'vp_pc': 'Clean price',
    'acc_int': 'InterestHistory'
}).reset_index()

In [69]:
TransformFintoolReferenceData = TransformFintoolReferenceData.merge(
    RUPriceHistoryLast[['isincode', 'Clean price pct', 'Clean price', 'InterestHistory']], how='left', left_on="ISIN", right_on="isincode"
).drop(columns=["isincode"])
TransformFintoolReferenceData["Interest"] = TransformFintoolReferenceData["Interest"].combine_first(TransformFintoolReferenceData["InterestHistory"])
TransformFintoolReferenceData["Interest"].fillna(0, inplace=True)
TransformFintoolReferenceData.drop(columns="InterestHistory", inplace=True)

In [70]:
bonds_index = TransformFintoolReferenceData[TransformFintoolReferenceData["Type"]=='Облигация'].index

In [71]:
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Price"] = \
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Clean price"] + \
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Interest"]

TransformFintoolReferenceData.loc[~TransformFintoolReferenceData.index.isin(bonds_index), "Price"] = \
TransformFintoolReferenceData.loc[~TransformFintoolReferenceData.index.isin(bonds_index), "Clean price"]

In [72]:
Prices.update(TransformFintoolReferenceData.groupby("ISIN").first())

In [73]:
exch_prices = EndOfDay.merge(
    ExchangeTree[['id', 'fullname_rus']], how='left', left_on='id_trade_site', right_on='id')[['fintoolId', 'last', 'fullname_rus']]
exch_prices["Source"] = exch_prices['fullname_rus'].apply(lambda x: 'RU Data (' + str(x) + ')')
exch_prices = exch_prices\
.rename(columns={
    'fintoolId': 'Fintool',
    'last': 'PriceEX'
})\
.drop(columns="fullname_rus")
exch_prices = exch_prices.merge(TransformFintoolReferenceData[["Fintool", "ISIN", "Type", "Principal", "Interest"]], how='inner')

In [74]:
exch_bonds = exch_prices[exch_prices['Type']=='Облигация']
exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
exch_bonds["Price"] = exch_bonds["Clean price"] + exch_bonds["Interest"]

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_27627/1720937637.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_27627/1720937637.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_27627/1720937637.py:4: SettingWithCopyWarning: 
A value

In [75]:
exch_not_bonds = exch_prices[exch_prices['Type']!='Облигация']
exch_not_bonds['Clean price'] = exch_not_bonds['PriceEX']
exch_not_bonds['Price'] = exch_not_bonds['PriceEX']

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_27627/3772375966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_not_bonds['Clean price'] = exch_not_bonds['PriceEX']
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_27627/3772375966.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_not_bonds['Price'] = exch_not_bonds['PriceEX']


In [76]:
Prices.update(exch_bonds.set_index("ISIN"))
Prices.update(exch_not_bonds.set_index("ISIN"))

In [77]:
ResultPrices = Prices[~Prices['Price'].isna()]
Prices = Prices[Prices['Price'].isna()]

#### Шаг 2. Cbonds

##### Шаг 2.1. Bonds

In [78]:
tradings_bonds = pd.read_excel(f'./data/Input/tradings_bonds.xls')
tradings_bonds_1 = pd.read_excel(f'./data/Input/tradings_bonds_1.xls')

tradings_bonds = pd.concat([tradings_bonds, tradings_bonds_1], axis=0)
tradings_bonds = tradings_bonds[['ISIN', 'Краткое название эмитента (rus)', 'Валюта', 'Индикативная цена', 'Номинал по еврооблигациям']]\
.rename(columns={
    'Краткое название эмитента (rus)': 'Name', 
    'Валюта': 'Currency', 
    'Индикативная цена': 'Clean price pct', 
    'Номинал по еврооблигациям': 'Principal EB'
})

In [79]:
tradings_bonds = tradings_bonds.merge(Prices[["Interest", "Principal"]], how='inner', left_on="ISIN", right_index=True)
principal_idx = tradings_bonds[tradings_bonds['Principal']>tradings_bonds['Principal EB']].index
tradings_bonds.loc[principal_idx, 'Principal'] /= tradings_bonds.loc[principal_idx, 'Principal EB']
tradings_bonds.loc[principal_idx, 'Interest'] /= tradings_bonds.loc[principal_idx, 'Principal']

In [80]:
Prices.update(tradings_bonds.set_index("ISIN"))

In [81]:
tradings_nkd = pd.read_excel(f'./data/Input/tradings_nkd.xls')
tradings_nkd_1 = pd.read_excel(f'./data/Input/tradings_nkd_1.xls')
tradings_nkd_2 = pd.read_excel(f'./data/Input/tradings_nkd_2.xls')
tradings_nkd_3 = pd.read_excel(f'./data/Input/tradings_nkd_3.xls')

tradings_nkd = pd.concat([tradings_nkd, tradings_nkd_1, tradings_nkd_2, tradings_nkd_3], axis=0)
tradings_nkd = tradings_nkd[['ISIN код эмиссии', 'Накопленный купонный доход', 'Непогашенный номинал']]\
.rename(columns={
    'ISIN код эмиссии': "ISIN", 
    'Накопленный купонный доход': "Interest", 
    'Непогашенный номинал': "Principal"
})
tradings_nkd = tradings_nkd.groupby("ISIN").first()

In [82]:
Prices.update(tradings_nkd)

In [83]:
tradings_new = pd.read_excel(f'./data/Input/tradings_new_{REPORT_DATE_STR}.xls')

tradings_new = tradings_new[['Код ISIN', 'Эмиссия', 'Дата', 'Bid, закр.', 'НКД']]\
.rename(columns={
    'Код ISIN': 'ISIN',
    'Эмиссия': 'Name',
    'Дата': 'Date',
    'Bid, закр.': 'Clean price pct',
    'НКД': 'Interest'
})
tradings_new["Max date"] = tradings_new.groupby('ISIN')["Date"].transform('max')
tradings_new = tradings_new.groupby('ISIN').first()

In [84]:
emissions = pd.read_excel(f'./data/Input/emissions_{REPORT_DATE_STR}.xls')

emissions = emissions[['Валюта', 'код ISIN', 'Непогашенный номинал']]\
.rename(columns={
    'код ISIN': 'ISIN',
    'Валюта': 'Currency',
    'Непогашенный номинал': 'Principal'
})
emissions = emissions[~((emissions["ISIN"].isna()) | (emissions["Principal"].isna()) | (emissions["Principal"]==0))].set_index("ISIN")
tradings_new = tradings_new.merge(emissions, how="left", left_index=True, right_index=True)

In [85]:
Prices.update(tradings_new)

In [86]:
Prices.loc[Prices['Interest'] > 0.2*Prices['Principal'], 'Interest'] = 0
Prices['Clean price'] = Prices['Clean price pct']/100 * Prices['Principal']
Prices['Price'] = Prices['Clean price'] + Prices['Interest']

In [87]:
cbonds_bonds_idx = Prices[~Prices["Price"].isna()].index
Prices.loc[cbonds_bonds_idx, "Type"] = 'Облигация'
Prices.loc[cbonds_bonds_idx, 'Source'] = 'Cbonds'
Prices.loc[cbonds_bonds_idx, 'ISIN in DB'] = 1

In [88]:
ResultPrices = pd.concat([ResultPrices, Prices.loc[cbonds_bonds_idx]], axis=0)
Prices = Prices[Prices['Price'].isna()]

##### Шаг 2.2. Stocks

In [89]:
tradings_stocks_all = pd.read_excel(f'./data/Input/tradings_stocks_all.xls')

tradings_stocks_all = tradings_stocks_all[['ISIN', 'Краткое название эмитента (rus)', 'Валюта номинала', 'Валюта торгов', 'Цена закрытия']]\
.rename(columns={
    'Краткое название эмитента (rus)': 'Name', 
    'Валюта номинала': 'Currency', 
    'Валюта торгов': 'Trade currency', 
    'Цена закрытия': 'Price'
})

In [90]:
tradings_stocks_all = tradings_stocks_all.merge(CurrencyRate[["currency", "rate"]], how='left', left_on="Trade currency", right_on="currency")\
.rename(columns={'rate': 'trade rate'})
tradings_stocks_all = tradings_stocks_all.merge(CurrencyRate[["currency", "rate"]], how='left', left_on="Currency", right_on="currency")
tradings_stocks_all['RUB Price'] = tradings_stocks_all['Price'] * tradings_stocks_all['trade rate']
tradings_stocks_all['RUB Price'] = tradings_stocks_all.groupby("ISIN")['RUB Price'].transform('mean')
tradings_stocks_all['Currency'] = tradings_stocks_all['Currency'].combine_first(tradings_stocks_all['Trade currency'])
tradings_stocks_all['Clean price'] = tradings_stocks_all['RUB Price'] / tradings_stocks_all['rate']
tradings_stocks_all['Price'] = tradings_stocks_all['Clean price']
tradings_stocks_all['FX Rate'] = tradings_stocks_all['trade rate'].combine_first(tradings_stocks_all['rate'])

In [91]:
cbonds_stocks_idx = tradings_stocks_all[~tradings_stocks_all["Price"].isna()].index
tradings_stocks_all.loc[cbonds_stocks_idx, "Type"] = 'Акция'
tradings_stocks_all.loc[cbonds_stocks_idx, 'Source'] = 'Cbonds'
tradings_stocks_all.loc[cbonds_stocks_idx, 'ISIN in DB'] = 1

In [92]:
Prices.update(tradings_stocks_all.groupby("ISIN").first())

In [93]:
ResultPrices = pd.concat([ResultPrices, Prices[~Prices["Price"].isna()]], axis=0)
Prices = Prices[Prices['Price'].isna()]

#### Шаг 3. MOEX

##### Шаг 3.0. RuData MOEX stage

In [94]:
HistoryStockBonds = RuDataDF("HistoryStockBonds").df
HistoryStockBonds.head()

HistoryStockBonds started
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
post processing
sleep 1
sleep 1
post processing
sleep 1
5000, 0
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
5000, 5000
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
5000, 10000
post start
post start
post start
post start
post start
post processing
post processing
post processing
sleep 1
post processing
sleep 1
sleep 1
sleep 1
post processing
sleep 1
5000, 15000
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
post processing
sleep 1
sleep 1
post processing
sleep 1
post processing
sleep 1
5000, 20000
post start
post start
post start
post start


,boardname,regnumber,isin,matdate,listname,facevalue,currencyid,volume,value,numtrades,prev,prevlegalcloseprice,openperiod,legalopenprice,legalcloseprice,closeperiod,waprice,trendclose,trendclspr,trendwap,trendwappr,openval,closeval,highbid,lowoffer,bid,offer,yieldatwap,yieldclose,accint,duration,marketprice,marketprice2,admittedquote,issuesize,mpvaltrd,mp2valtrd,admittedvalue,marketprice3,marketprice3tradesvalue,type,lastprice,iricpiclose,beiclose,couponpercent,couponvalue,buybackdate,lasttradedate,cbrclose,yieldtooffer,yieldlastcoupon,offerdate,faceunit,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,Т+: Облигации - безадрес.,4-01-36388-R,RU000A0JRU20,2031-09-26T00:00:00,3,709.0,SUR,4,2574.52,1,90.78,90.78,NaN,None,90.78,None,90.78,NaN,NaN,-0.04,-0.04,2574.52,2574.52,90.78,90.78,89.00,90.87,131.37,131.37,27.62,46.0,None,NaN,None,5000000,0.0,0.0,None,90.68,747039.46,corporate_bond,90.78,NaN,NaN,10.69,37.79,2024-10-04T00:00:00,2024-08-16T00:00:00,None,NaN,NaN,None,RUB,stock,bonds,TQCB,2,90.78,90.78,90.78,90.78,RU000A0JRU20,СЗКК 03,2024-08-16T00:00:00,1,NaN
1,Т+: Облигации - безадрес.,4-01-36388-R,RU000A0JRU20,2031-09-26T00:00:00,3,709.0,SUR,0,0.00,0,90.78,90.78,NaN,None,90.78,None,NaN,NaN,NaN,NaN,NaN,0.00,0.00,89.00,90.91,89.00,90.92,NaN,NaN,28.24,45.0,None,NaN,None,5000000,0.0,0.0,None,90.68,747039.46,corporate_bond,NaN,NaN,NaN,10.69,37.79,2024-10-04T00:00:00,2024-08-16T00:00:00,None,NaN,NaN,None,RUB,stock,bonds,TQCB,2,NaN,NaN,NaN,NaN,RU000A0JRU20,СЗКК 03,2024-08-19T00:00:00,1,NaN
2,Т+: Облигации - безадрес.,4-01-36388-R,RU000A0JRU20,2031-09-26T00:00:00,3,709.0,SUR,3,1917.57,3,90.78,90.78,NaN,None,89.00,None,90.15,-1.78,-1.96,-0.63,-0.69,643.28,631.01,90.78,90.72,88.21,90.73,152.72,179.83,28.45,44.0,None,NaN,None,5000000,0.0,0.0,None,90.67,748957.03,corporate_bond,89.00,NaN,NaN,10.69,37.79,2024-10-04T00:00:00,2024-08-20T00:00:00,None,NaN,NaN,None,RUB,stock,bonds,TQCB,2,90.73,89.00,90.73,89.00,RU000A0JRU20,СЗКК 03,2024-08-20T00:00:00,1,NaN
3,Т+: Облигации - безадрес.,4-01-36388-R,RU000A0JRU20,2031-09-26T00:00:00,3,709.0,SUR,0,0.00,0,89.00,89.00,NaN,None,89.00,None,NaN,NaN,NaN,NaN,NaN,0.00,0.00,88.21,90.62,88.21,90.63,NaN,NaN,28.66,43.0,None,NaN,None,5000000,0.0,0.0,None,90.67,748957.03,corporate_bond,NaN,NaN,NaN,10.69,37.79,2024-10-04T00:00:00,2024-08-20T00:00:00,None,NaN,NaN,None,RUB,stock,bonds,TQCB,2,NaN,NaN,NaN,NaN,RU000A0JRU20,СЗКК 03,2024-08-21T00:00:00,1,NaN
4,Т+: Облигации - безадрес.,4-01-36388-R,RU000A0JRU20,2031-09-26T00:00:00,3,709.0,SUR,3,1928.84,2,89.00,89.00,NaN,None,90.71,None,90.68,1.71,1.92,0.53,0.59,642.57,1286.27,90.63,90.62,88.21,90.69,150.22,149.54,28.86,42.0,None,NaN,None,5000000,0.0,0.0,None,90.67,750885.87,corporate_bond,90.71,NaN,NaN,10.69,37.79,2024-10-04T00:00:00,2024-08-22T00:00:00,None,NaN,NaN,None,RUB,stock,bonds,TQCB,2,90.71,90.63,90.63,90.71,RU000A0JRU20,СЗКК 03,2024-08-22T00:00:00,1,NaN


In [95]:
HistoryStockShares = RuDataDF("HistoryStockShares").df
HistoryStockShares.head()

HistoryStockShares started
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
post processing
sleep 1
5000, 0
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
5000, 5000
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
sleep 1
post processing
sleep 1
sleep 1
sleep 1
sleep 1
5000, 10000
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
5000, 15000
post start
post start
post start
post start
post start
post processing
post processing
sleep 1
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
5000, 20000
post start
post start
post start
post start

,boardname,regnumber,isin,listname,facevalue,currencyid,prevlegalcloseprice,prev,legalopenprice,openperiod,closeperiod,openval,closeval,legalcloseprice,trendclose,trendclspr,highbid,bid,offer,lowoffer,waprice,trendwap,trendwappr,volume,marketprice,marketprice2,mp2valtrd,mpvaltrd,value,numtrades,issuesize,admittedquote,admittedvalue,monthlycapitalization,dailycapitalization,marketprice3,marketprice3tradesvalue,type,closeauctionprice,waval,lastprice,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,Т+: Акции и ДР - безадрес.,1-01-00283-A,RU0009100911,3,0.25,SUR,7440.0,7440.0,None,NaN,None,0.0,0.0,7440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,None,NaN,0.0,0.0,0.0,0,398272,None,None,NaN,NaN,NaN,0.0,common_share,7440.0,0.0,NaN,stock,shares,TQBR,0,NaN,NaN,NaN,NaN,MGNZ,СМЗ-ао,2024-08-28T00:00:00,3,NaN
1,Т+: Акции и ДР - безадрес.,1-01-00283-A,RU0009100911,3,0.25,SUR,7440.0,7440.0,None,NaN,None,0.0,0.0,7440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,None,NaN,0.0,0.0,0.0,0,398272,None,None,NaN,NaN,NaN,0.0,common_share,7440.0,0.0,NaN,stock,shares,TQBR,0,NaN,NaN,NaN,NaN,MGNZ,СМЗ-ао,2024-08-29T00:00:00,3,NaN
2,Т+: Акции и ДР - безадрес.,1-01-00283-A,RU0009100911,3,0.25,SUR,7440.0,7440.0,None,NaN,None,0.0,0.0,7440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,None,NaN,0.0,0.0,0.0,0,398272,None,None,NaN,NaN,NaN,0.0,common_share,7440.0,0.0,NaN,stock,shares,TQBR,0,NaN,NaN,NaN,NaN,MGNZ,СМЗ-ао,2024-08-30T00:00:00,3,NaN
3,Т+: Акции и ДР - безадрес.,1-01-00283-A,RU0009100911,3,0.25,SUR,7440.0,7440.0,None,NaN,None,0.0,0.0,7440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,None,NaN,0.0,0.0,0.0,0,398272,None,None,NaN,NaN,NaN,0.0,common_share,7440.0,0.0,NaN,stock,shares,TQBR,0,NaN,NaN,NaN,NaN,MGNZ,СМЗ-ао,2024-07-31T00:00:00,1,NaN
4,Т+: Акции и ДР - безадрес.,1-01-00283-A,RU0009100911,3,0.25,SUR,7440.0,7440.0,None,NaN,None,0.0,0.0,7440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,None,NaN,0.0,0.0,0.0,0,398272,None,None,NaN,NaN,NaN,0.0,common_share,7440.0,0.0,NaN,stock,shares,TQBR,0,NaN,NaN,NaN,NaN,MGNZ,СМЗ-ао,2024-08-01T00:00:00,1,NaN


In [96]:
HistoryStockNdm = RuDataDF("HistoryStockNdm").df
HistoryStockNdm.head()

HistoryStockNdm started
post start
post start
post start
post start
post start
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
post processing
sleep 1
post processing
sleep 1
5000, 0
post start
post start
post start
post start
post start
post processing
post processing
post processing
sleep 1
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
5000, 5000
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
5000, 10000
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
post processing
sleep 1
5000, 15000
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
766, 20000
post start
post start
post start
post start
pos

,boardname,closeval,currencyid,facevalue,isin,issuesize,listname,matdate,numtrades,openval,prev,regnumber,trendclose,trendclspr,trendwap,trendwappr,type,value,volume,waprice,yieldatwap,yieldclose,lastprice,iricpiclose,beiclose,couponpercent,couponvalue,buybackdate,lasttradedate,cbrclose,yieldtooffer,yieldlastcoupon,offerdate,faceunit,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,РПС с ЦК: Облигации - адрес.,0.0,SUR,100000.0,RU000A107A69,84379,3,2025-08-04T00:00:00,0,0.0,90.90,4-09-01326-B,NaN,NaN,NaN,NaN,corporate_bond,0.0,0,NaN,NaN,NaN,None,NaN,NaN,7.85,3925.0,None,2024-06-13T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PTOB,2,NaN,NaN,NaN,NaN,RU000A107A69,АЛЬФАБ ЗО2,2024-08-27T00:00:00,2,NaN
1,РПС: Облигации - адрес.,0.0,SUR,100000.0,RU000A107A69,84379,3,2025-08-04T00:00:00,0,0.0,90.95,4-09-01326-B,NaN,NaN,NaN,NaN,corporate_bond,0.0,0,NaN,NaN,NaN,None,NaN,NaN,7.85,3925.0,None,2024-06-13T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PSOB,2,NaN,NaN,NaN,NaN,RU000A107A69,АЛЬФАБ ЗО2,2024-08-28T00:00:00,2,NaN
2,РПС с ЦК: Облигации - адрес.,0.0,SUR,100000.0,RU000A107A69,84379,3,2025-08-04T00:00:00,0,0.0,90.90,4-09-01326-B,NaN,NaN,NaN,NaN,corporate_bond,0.0,0,NaN,NaN,NaN,None,NaN,NaN,7.85,3925.0,None,2024-06-13T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PTOB,2,NaN,NaN,NaN,NaN,RU000A107A69,АЛЬФАБ ЗО2,2024-08-28T00:00:00,2,NaN
3,РПС: Облигации - адрес.,0.0,SUR,100000.0,RU000A107A69,84379,3,2025-08-04T00:00:00,0,0.0,90.95,4-09-01326-B,NaN,NaN,NaN,NaN,corporate_bond,0.0,0,NaN,NaN,NaN,None,NaN,NaN,7.85,3925.0,None,2024-06-13T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PSOB,2,NaN,NaN,NaN,NaN,RU000A107A69,АЛЬФАБ ЗО2,2024-08-29T00:00:00,2,NaN
4,РПС с ЦК: Облигации - адрес.,0.0,SUR,100000.0,RU000A107A69,84379,3,2025-08-04T00:00:00,0,0.0,90.90,4-09-01326-B,NaN,NaN,NaN,NaN,corporate_bond,0.0,0,NaN,NaN,NaN,None,NaN,NaN,7.85,3925.0,None,2024-06-13T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PTOB,2,NaN,NaN,NaN,NaN,RU000A107A69,АЛЬФАБ ЗО2,2024-08-29T00:00:00,2,NaN


In [97]:
HistoryStockCcp = RuDataDF("HistoryStockCcp").df
HistoryStockCcp.head()

HistoryStockCcp started
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
post processing
sleep 1
5000, 0
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
post processing
sleep 1
5000, 5000
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
5000, 10000
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
5000, 15000
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
5000, 20000
post start
post start
post start
post start
po

,accint,admittedquote,admittedvalue,bid,boardname,closeperiod,closeval,currencyid,duration,facevalue,highbid,isin,issuesize,legalcloseprice,legalopenprice,listname,lowoffer,marketprice,marketprice2,marketprice3,marketprice3tradesvalue,matdate,mp2valtrd,mpvaltrd,numtrades,offer,openperiod,openval,prev,prevlegalcloseprice,regnumber,trendclose,trendclspr,trendwap,trendwappr,type,value,volume,waprice,yieldatwap,yieldclose,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,1488.3403,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,7.997030e+07,SUR,253.0,1000.0,17.05,RU000A1078V2,292090,None,None,3,17.05,17.14,None,None,0.0,2025-04-17T00:00:00,0.0,0.0,3,None,None,1.052625e+07,17.10,None,4-04-32432-H,-0.05,None,-0.05,NaN,corporate_bond,1.362565e+08,1864,17.05,None,None,stock,ccp,EQRP,4,17.05,17.05,17.05,17.05,RU000A1078V2,ГТЛК ЗО25Д,2024-08-02T00:00:00,1,NaN
1,1488.3403,None,None,None,РЕПО с ЦК - адрес.,None,9.502869e+08,SUR,NaN,1000.0,NaN,RU000A1078V2,292090,None,None,3,NaN,17.14,None,None,0.0,2025-04-17T00:00:00,0.0,0.0,1,None,None,9.502869e+08,17.30,None,4-04-32432-H,-0.15,None,NaN,NaN,corporate_bond,9.502869e+08,13000,NaN,None,None,stock,ccp,PSRP,4,17.15,17.15,17.15,17.15,RU000A1078V2,ГТЛК ЗО25Д,2024-08-02T00:00:00,1,NaN
2,1529.7878,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,1.377234e+08,SUR,250.0,1000.0,17.10,RU000A1078V2,292090,None,None,3,17.10,17.10,None,None,0.0,2025-04-17T00:00:00,0.0,0.0,1,None,None,1.377234e+08,17.05,None,4-04-32432-H,0.05,None,0.05,NaN,corporate_bond,1.377234e+08,1884,17.10,None,None,stock,ccp,EQRP,4,17.10,17.10,17.10,17.10,RU000A1078V2,ГТЛК ЗО25Д,2024-08-05T00:00:00,1,NaN
3,1529.7878,None,None,None,РЕПО с ЦК - адрес.,None,9.503210e+08,SUR,NaN,1000.0,NaN,RU000A1078V2,292090,None,None,3,NaN,17.10,None,None,0.0,2025-04-17T00:00:00,0.0,0.0,1,None,None,9.503210e+08,17.15,None,4-04-32432-H,NaN,None,NaN,NaN,corporate_bond,9.503210e+08,13000,NaN,None,None,stock,ccp,PSRP,4,17.15,17.15,17.15,17.15,RU000A1078V2,ГТЛК ЗО25Д,2024-08-05T00:00:00,1,NaN
4,1530.7467,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,1.343634e+08,SUR,249.0,1000.0,17.10,RU000A1078V2,292090,None,None,3,17.10,17.24,None,None,0.0,2025-04-17T00:00:00,0.0,0.0,1,None,None,1.343634e+08,17.10,None,4-04-32432-H,NaN,None,NaN,NaN,corporate_bond,1.343634e+08,1855,17.10,None,None,stock,ccp,EQRP,4,17.10,17.10,17.10,17.10,RU000A1078V2,ГТЛК ЗО25Д,2024-08-06T00:00:00,1,NaN


##### Шаг 3.1. Tranform

In [98]:
TransformHistoryStockBonds = HistoryStockBonds.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockShares = HistoryStockShares.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockNdm = HistoryStockNdm.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockCcp = HistoryStockCcp.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")

In [99]:
Prices.update(TransformHistoryStockBonds)
Prices.update(TransformHistoryStockShares)
Prices.update(TransformHistoryStockNdm)
Prices.update(TransformHistoryStockCcp)

In [100]:
moex_idx = Prices[~Prices["Price"].isna()].index
Prices.loc[moex_idx, 'Source'] = 'RU Data (MOEX)'
Prices.loc[moex_idx, 'ISIN in DB'] = 1

In [101]:
ResultPrices = pd.concat([ResultPrices, Prices.loc[moex_idx]], axis=0)
Prices = Prices[Prices['Price'].isna()]

#### Шаг 4. Finish

In [102]:
FullPrices = pd.concat([ResultPrices, Prices], axis=0)

##### Шаг 4.1. Update currency

In [103]:
FullPrices = FullPrices\
.reset_index()\
.merge(CurrencyRate[['currency', 'rate']], how='left', left_on='Currency', right_on='currency')\
.set_index("ISIN")

In [104]:
rub_price_idx = FullPrices[FullPrices["RUB Price"].isna()].index
FullPrices.loc[rub_price_idx, "FX Rate"] = FullPrices.loc[rub_price_idx, "rate"].fillna(1)
FullPrices.loc[rub_price_idx, "RUB Price"] = FullPrices.loc[rub_price_idx, "Price"] * FullPrices.loc[rub_price_idx, "FX Rate"]

In [105]:
FullPrices.drop(columns=['currency', 'rate'], inplace=True)

##### Шаг 4.2. Write excel

Добавление времени выгрузки

In [ ]:
FullPrices["export_timestamp"] = dt.now().strftime("%Y%m%d%H%M%S")

In [108]:
with pd.ExcelWriter(f'./data/Output/prices.xlsx', datetime_format='dd.mm.yyyy') as writer: 
    FullPrices[~FullPrices["Price"].isna()].to_excel(writer)

In [109]:
with pd.ExcelWriter(f'./data/Output/prices_full.xlsx', datetime_format='dd.mm.yyyy') as writer: 
    FullPrices.to_excel(writer)

In [ ]:
from functions.send_email import main
main("prices.xlsx", "prices_full.xlsx")